## 3. Data Preparation
📒 `2.0-rc-data-preparation.ipynb`

**Objetivo:** Transformar os dados brutos em um formato que possa ser usado para modelagem.

⚙️ Aqui você:
- Limpeza dos Dados, identificados na fase anterior
- Trata dados faltantes, padroniza formatos, remove duplicatas
- Faz conversões de tipos, normalizações, e categorização
- Gera novas features e refina o dataset para o modelo
- - Salvamento do dataset limpo (`data/processed/`)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

